In [24]:
import cv2
import tensorflow as tf
from imutils import paths
from matplotlib import pyplot as plt
import random
import re
import pickle
import os
from mtcnn.mtcnn import MTCNN
from numpy import asarray
from PIL import Image
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from keras_vggface.utils import preprocess_input
from keras_vggface.vggface import VGGFace
from scipy.spatial.distance import cosine
from sklearn.svm import SVC

In [3]:
def extract_face_from_image(image_path, required_size=(224, 224)):
  # load image and detect faces
    image = plt.imread(image_path)
    detector = MTCNN()
    faces = detector.detect_faces(image)

    face_images = []

    for face in faces:
        # extract the bounding box from the requested face
        x1, y1, width, height = face['box']
        x2, y2 = x1 + width, y1 + height

        # extract the face
        face_boundary = image[y1:y2, x1:x2]

        # resize pixels to the model size
        face_image = Image.fromarray(face_boundary)
        face_image = face_image.resize(required_size)
        face_array = asarray(face_image)
        face_images.append(face_array)

    return face_images

In [4]:
def get_model_scores(faces):
    samples = asarray(faces, 'float32')
    samples = np.array(samples)
    samples=samples.squeeze()
    # prepare the data for the model
    samples = preprocess_input(samples, version=2)

    # create a vggface model object
    model = VGGFace(model='resnet50',
      include_top=False,
      input_shape=(224, 224, 3),
      pooling='avg')

    # perform prediction
    return model(samples)

#### getting images

In [56]:
imagePaths = list(paths.list_images('image_dataset'))

In [57]:
for i in imagePaths:
    print(i)

image_dataset\archit\opencv_frame_1.jpg
image_dataset\archit\opencv_frame_2.jpg
image_dataset\archit\opencv_frame_3.jpg
image_dataset\archit\opencv_frame_4.jpg
image_dataset\archit\opencv_frame_5.jpg
image_dataset\archit\opencv_frame_6.jpg
image_dataset\divayanshu\image_0.jpg
image_dataset\divayanshu\photo_3.jpg
image_dataset\navneet\1.jpeg
image_dataset\navneet\2.jpeg
image_dataset\navneet\3.jpeg
image_dataset\navneet\4.jpeg
image_dataset\navneet\5.jpeg
image_dataset\navneet\6.jpeg
image_dataset\unknown\1.jpg
image_dataset\unknown\10.jpg
image_dataset\unknown\11.jpg
image_dataset\unknown\12.jpg
image_dataset\unknown\13.jpg
image_dataset\unknown\14.jpg
image_dataset\unknown\15.jpg
image_dataset\unknown\16.jpg
image_dataset\unknown\18.jpg
image_dataset\unknown\19.jpg
image_dataset\unknown\2.jpg
image_dataset\unknown\20.jpg
image_dataset\unknown\21.jpg
image_dataset\unknown\23.jpg
image_dataset\unknown\24.jpg
image_dataset\unknown\25.jpg
image_dataset\unknown\26.jpg
image_dataset\unknown

#### getting features

In [58]:
faces = [extract_face_from_image(image_path)
         for image_path in imagePaths]

In [59]:
model_scores = get_model_scores(faces)

In [60]:
np.shape(model_scores)

TensorShape([43, 2048])

#### SVM Model

In [61]:
# converting into ndarray
model_score_arr=model_scores.numpy()

In [62]:
model_score_arr

array([[0.0000000e+00, 4.0722494e+00, 1.8815443e-01, ..., 1.4792593e+00,
        0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 2.4249964e+00, 2.0735122e-01, ..., 2.0060418e+00,
        0.0000000e+00, 0.0000000e+00],
       [4.3977816e-03, 2.9801433e+00, 2.3178743e-01, ..., 1.2829052e+00,
        0.0000000e+00, 0.0000000e+00],
       ...,
       [7.3319459e+00, 1.0658642e+00, 0.0000000e+00, ..., 7.5559694e-01,
        1.3588525e-02, 6.3880005e+00],
       [3.4013638e+00, 1.4701334e+00, 0.0000000e+00, ..., 0.0000000e+00,
        1.8825968e-01, 6.0273118e+00],
       [0.0000000e+00, 0.0000000e+00, 9.5242012e-01, ..., 6.3157468e+00,
        0.0000000e+00, 0.0000000e+00]], dtype=float32)

In [63]:
labels=[]
for imagePath in imagePaths:
        label = imagePath.split(os.path.sep)[-2]
        labels.append(label)

In [64]:
labels

['archit',
 'archit',
 'archit',
 'archit',
 'archit',
 'archit',
 'divayanshu',
 'divayanshu',
 'navneet',
 'navneet',
 'navneet',
 'navneet',
 'navneet',
 'navneet',
 'unknown',
 'unknown',
 'unknown',
 'unknown',
 'unknown',
 'unknown',
 'unknown',
 'unknown',
 'unknown',
 'unknown',
 'unknown',
 'unknown',
 'unknown',
 'unknown',
 'unknown',
 'unknown',
 'unknown',
 'unknown',
 'unknown',
 'unknown',
 'unknown',
 'unknown',
 'unknown',
 'unknown',
 'unknown',
 'unknown',
 'unknown',
 'unknown',
 'unknown']

In [65]:
classifier = SVC(kernel='rbf', random_state = 1,probability=True)

In [66]:
classifier.fit(model_score_arr,labels)

SVC(probability=True, random_state=1)

Y_pred = classifier.predict(X_test)

#### test

In [28]:
#imagePaths_test = list(paths.list_images('image_dataset')

In [29]:
faces_test = [extract_face_from_image(image_path)
         for image_path in ['test_01.jpg','test_02.jpg','test_03.jpeg','16_01.jpeg','16_02.jpeg','16_03.jpeg','16_04.jpeg','16_05.jpeg','16_06.jpeg','16_07.jpeg','16_08.jpeg','16_09.jpeg','16_10.jpeg','16_11.jpeg','16_12.jpeg','16_13.jpeg','16_14.jpg','16_15.jpg','16_16.jpg','16_17.jpg','16_18.jpg','16_19.jpg','16_20.jpg','16_21.jpg','16_22.jpeg','16_23.jpeg','16_24.jpeg','16_25.jpeg','16_26.jpeg']]

In [30]:
model_scores_test = get_model_scores(faces_test)

In [31]:
# converting into ndarray
model_score_test_arr=model_scores_test.numpy()

In [32]:
np.shape(model_score_test_arr)

(29, 2048)

In [67]:
Y_pred = classifier.predict(model_score_test_arr)

In [68]:
Y_pred

array(['unknown', 'navneet', 'archit', 'unknown', 'unknown', 'navneet',
       'unknown', 'archit', 'unknown', 'navneet', 'unknown', 'unknown',
       'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown',
       'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown',
       'navneet', 'navneet', 'unknown', 'navneet', 'navneet'],
      dtype='<U10')

In [69]:
Y_pred_1 = classifier.predict_proba(model_score_test_arr) 

In [70]:
Y_pred_1

array([[0.02500978, 0.03596519, 0.01942554, 0.9195995 ],
       [0.06433666, 0.05007829, 0.22740293, 0.65818212],
       [0.65410752, 0.00662229, 0.04532104, 0.29394916],
       [0.02880866, 0.00591062, 0.02632688, 0.93895384],
       [0.04721641, 0.03383075, 0.15784571, 0.76110713],
       [0.05569497, 0.0633756 , 0.54533506, 0.33559436],
       [0.05064806, 0.03290628, 0.16203991, 0.75440575],
       [0.2974873 , 0.00806215, 0.03679461, 0.65765594],
       [0.03118257, 0.00261994, 0.02725679, 0.9389407 ],
       [0.05964345, 0.05304797, 0.24045905, 0.64684952],
       [0.04652688, 0.0657888 , 0.20170052, 0.6859838 ],
       [0.14208572, 0.00682308, 0.02932309, 0.82176812],
       [0.03506942, 0.01960695, 0.08218162, 0.86314201],
       [0.05749415, 0.03542441, 0.18349286, 0.72358858],
       [0.02891128, 0.01823957, 0.05888486, 0.89396429],
       [0.0880352 , 0.00745466, 0.02368302, 0.88082711],
       [0.02437159, 0.01365325, 0.02193972, 0.94003544],
       [0.03003394, 0.00835145,